S&P500 stock data:
http://pages.swcp.com/stocks/


n = pd.read_csv('http://chart.finance.yahoo.com/table.csv?s=ACIU&a=7&b=9&c=1996&d=10&e=19&f=2016&g=d&ignore=.csv')

http://www.eoddata.com/download.aspx

In [267]:
from sqlalchemy.orm import sessionmaker
from sqlalchemy import Table
from sqlalchemy import create_engine, inspect
from sqlalchemy import MetaData
from edgerdb import helper_functions as hlp
import pandas as pd
import numpy as np
import os


meta = MetaData()

engine = create_engine('postgresql://analyst:@localhost:5432/edgar')

messages = Table('filings', meta, autoload=True, autoload_with=engine)

def sql_to_df(query, connection, index=None):
    df = pd.read_sql(query, connection)
    connection.close()
    return df

In [2]:
cik_and_tickers = sql_to_df('select * from temp_cik_to_ticker;', engine.connect()).set_index(keys='index')

In [3]:
cik_and_tickers.head()

,cik,company_name,exchange,isactive,symbol
index,,,,,
0,1591890,"1347 PROPERTY INSURANCE HOLDINGS, INC.",NASDAQ,1,PIH
1,1508475,"21VIANET GROUP, INC.",NASDAQ,1,VNET
2,1459417,"2U, INC.",NASDAQ,1,TWOU
3,1295484,"51JOB, INC.",NASDAQ,1,JOBS
4,1295484,"51JOB, INC.",NASDAQ,1,JOBS


In [4]:
tickers = cik_and_tickers['symbol'].tolist()


In [57]:
import requests
from yahoo_finance import Share
from pprint import pprint

In [58]:
yahoo = Share('YHOO')
print(yahoo.get_open())
print(yahoo.get_price())
pprint(yahoo.get_historical('2014-04-25', '2014-04-29'))

None
41.45
[{'Adj_Close': '35.830002',
  'Close': '35.830002',
  'Date': '2014-04-29',
  'High': '35.889999',
  'Low': '34.119999',
  'Open': '34.369999',
  'Symbol': 'YHOO',
  'Volume': '28736000'},
 {'Adj_Close': '33.990002',
  'Close': '33.990002',
  'Date': '2014-04-28',
  'High': '35.00',
  'Low': '33.650002',
  'Open': '34.669998',
  'Symbol': 'YHOO',
  'Volume': '31019200'},
 {'Adj_Close': '34.48',
  'Close': '34.48',
  'Date': '2014-04-25',
  'High': '35.099998',
  'Low': '34.290001',
  'Open': '35.029999',
  'Symbol': 'YHOO',
  'Volume': '19401600'}]


In [97]:
# Columns will need to be added to top of file after download
sp500 = pd.read_csv('data/sp500hst.txt')

In [99]:
len(sp500['symbol'].unique())

524

In [191]:
tickers[0:10]

['PIH',
 'VNET',
 'TWOU',
 'JOBS',
 'JOBS',
 'CAFD',
 'ABEO',
 'ABEOW',
 'ABIL',
 'ACIU']

In [268]:
r2016 = pd.read_csv('data/5_YEAR_NASDAQ/NASDAQ_20160101.csv')

In [270]:
print(len(r2016['Symbol'].tolist()))
print(len(set(r2016['Symbol'].tolist() + tickers)))

2812
13534


In [288]:
def in_both(first_li, second_li):
    both = []
    for x in second_li:
        if x in first_li:
            both.append(x)
    return both


def find_csv_files_in_dir(directory):
    paths = []
    for subdir, dirs, files in os.walk(directory):
        for file in files:
            filepath = subdir + os.sep + file
            if filepath.endswith(".csv"):
                paths.append(filepath)
    return paths


def concat_eod_data(list_of_paths):
    '''
        This functions takes as input a list of paths to csv files containg eod stock data and join the files
        into a single dataframe.
    '''
    df = pd.DataFrame(columns=['Symbol', 'Date', 'Open', 'High', 'Low', 'Close', 'Volume'])
    for path in list_of_paths:
        temp_df = pd.read_csv(path)
        df = pd.concat([df, temp_df])
    return df

In [272]:
five_year_nasdaq_files = find_csv_files_in_dir('data/5_YEAR_NASDAQ')

In [ ]:
concat_eod_data(five_year_nasdaq_files)